In [ ]:
#! /usr/bin/env python

import requests, csv, time
MAX_RETRIES = 20

# In : Gidsvoornederland.nl URL to a public library infopage : e.g. https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199891
# Out: URL to Wayback Machine (The Internet Archive - TIA) snapshot: e.g. https://web.archive.org/web/20181111190849/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199891 
# Works for html pages in  Gidsvoornederland.nl
# This script is certainly not 100% solid & error proof, you might need to tweak it here and there to make it more robust and deal with all sorts of possible errors and time-outs. But hey, it does a mostly decent job!

def read_local_csv(filename):
    # Input=local filepath/filename of csv -- Output=List
    list = []
    response = open(filename, 'r', encoding="utf8")
    sheet = csv.reader(response, delimiter=';')
    for row in sheet:
        if not str(row).startswith('#', 2):  # skip rows in cvs that start with '#'
            list.append(row)
    return list

base_url = 'http://web.archive.org'

urllist = []
urllist=read_local_csv("Input-Gids_TeArchiverenURLs_11112018_werkbestand.txt") # file with 1 to-be-archived Leesplein-URL per line

with open("Output-Gids_TeArchiverenURLs_11112018_werkbestand.txt", "w") as outputfile: # open new versin of outputfile, clear the old version

    errorfile = open('Output-Gids_TeArchiverenURLs_11112018_werkbestand.txt', 'a') # deze file bevat alle LP-urls die in de 1e run niet gearchiveerd konden worden

    for url in urllist:
        time.sleep(5)  # pause for 5 sec, not to overrequest the server. Choose longer times for archiving pdf and doc files, as the archiving software needs more time to read these docs than standard html pages
        outputfile = open('Output-Gids_TeArchiverenURLs_11112018_werkbestand.txt', 'a') # append to existing outputfile
        linenumber = urllist.index(url)+1
        #print(str(url[0]))

        #https://stackoverflow.com/questions/33895739/python-requests-cant-load-any-url-remote-end-closed-connection-without-respo
        url_wbm = base_url + '/save/' + str(url[0])
        session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(max_retries=MAX_RETRIES)
        session.mount('https://', adapter)
        session.mount('http://', adapter)

        r = session.get(url_wbm)  #This line actually saves to URL to the Waybackmachine. All the following lines of code are for administration/display
        print(str(url[0])+ " -- " + str(r.headers))

# r.headers fout die kan voorkomen: {'Server': 'nginx/1.13.11', ...., 'X-Archive-Wayback-Runtime-Error': 'WaybackException: Unexpected Error', 'Set-Cookie': 'JSESSIONID=9EAAD47DBDCB85D0514293D472995036; Path=/; HttpOnly', 'X-App-Server': 'wwwb-app53', 'X-ts': '----'}

# Als deze fout voorkomt en de LvdL-URL daardoor niet gearchiveerd wordt: Schrijf niet gearchiveerde LvdL-url weg naar apart bestand - dat kunnen we dan later als invoer voor een correctierun gebruiken

        if 'X-Archive-Wayback-Runtime-Error' in r.headers:
            errorfile.write(str(url[0]) + "\n")
        else: #r.headers bevat geen Error
            #archive_url ="" #only relevant for pdf and doc
            #archive_url= r.headers['X-Cache-Key']
            # X-Cache-Key for PDF and DOC: httpweb.archive.org/web/20180608110403/https://www.leesplein.nl/assets/juryrapporten/2015-schaduw.pdfNL
            # X-Cache-Key for HTML: httpweb.archive.org/save/https://www.leesplein.nl/JB_plein.php?hm=3&sm=1&leefcat=12+&id=6726NL

            #if archive_url.startswith('httpweb.') and archive_url.endswith('NL'):
                #archive_url = "http://"+archive_url[4:-2]
                # for PDF and DOC : http://web.archive.org/web/20180608110403/https://www.leesplein.nl/assets/juryrapporten/2015-schaduw.pdf
                # for HTML: http://web.archive.org/save/https://www.leesplein.nl/JB_plein.php?hm=3&sm=1&leefcat=12+&id=6726
            #else: print('Unexpected value of X-Cache-Key in r.headers')

            if r.status_code == requests.codes.ok:
                contenttype = r.headers['Content-Type']
                if "text/html" in contenttype: #html page -- X-Cache-Key is not relevant
                    print(str(linenumber) + "^^^^" + str(url[0])+ "^^^^" + base_url + r.headers['Content-Location']) # the "^^^^" is used a separator between the two URLs. The more traditional separators such as ";" or "," are not reliable enough, as the can occur in URLs.
                    outputfile.write(str(linenumber) + "^^^^" + str(url[0])+ "^^^^" + base_url + r.headers['Content-Location'] + "\n")
                #elif contenttype == "application/pdf": #pdf file
                    #print(str(linenumber) + "^^^^" + str(url[0]) + "^^^^" + archive_url)
                    #outputfile.write(str(linenumber) + "^^^^" + str(url[0]) + "^^^^" + archive_url + "\n")
                #elif contenttype == "application/vnd.openxmlformats-officedocument.wordprocessingml.document": #Word doc(x)
                    #print(str(linenumber) + "^^^^" + str(url[0]) + "^^^^" + archive_url)
                    #outputfile.write(str(linenumber) + "^^^^" + str(url[0]) + "^^^^" + archive_url + "\n")
                else:
                    print("Unknown Content-Type: " + str(contenttype))
            else:
                print('Error in response: ' + str(r.status_code))
        outputfile.close()
        #errorfile.close()

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5295735&startindex=810 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:15:16 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112221515/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5295735&startindex=810', 'Set-Cookie': 'JSESSIONID=295CF429DA19890507950588B911C445; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pr

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5295753&startindex=810 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:15:50 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112221548/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5295753&startindex=810', 'Set-Cookie': 'JSESSIONID=855813511E1418C595FC9917669D4C87; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pr

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296447&startindex=820 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:16:23 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112221622/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5296447&startindex=820', 'Set-Cookie': 'JSESSIONID=33D222C4E4B5908FD923AEBBFD8A3F50; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pr

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5295673&startindex=820 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:16:56 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112221655/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5295673&startindex=820', 'Set-Cookie': 'JSESSIONID=8E63C04CBF437E79A53939E2C23984AE; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pr

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5295736&startindex=830 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:17:32 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112221731/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5295736&startindex=830', 'Set-Cookie': 'JSESSIONID=7666DF3230D80CCC467C5333A4A1E532; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pr

...

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199750&startindex=990 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:36:17 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112223616/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199750&startindex=990', 'Set-Cookie': 'JSESSIONID=0442C00C664E429488F711A53A01165B; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5320747&startindex=1000 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:36:58 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112223657/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=5320747&startindex=1000', 'Set-Cookie': 'JSESSIONID=1954A55B4775B70BED15BEBA55273E3E; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, 

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199978&startindex=1000 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:37:33 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112223732/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199978&startindex=1000', 'Set-Cookie': 'JSESSIONID=6D1AD8B7A13CF2C1A79B967210D96571; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pr

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199298&startindex=1010 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:38:07 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112223805/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199298&startindex=1010', 'Set-Cookie': 'JSESSIONID=6F7DD04364B173B6E98DACA6B82E561F; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pr

https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199398&startindex=1010 -- {'Server': 'nginx/1.15.5', 'Date': 'Mon, 12 Nov 2018 22:38:41 GMT', 'Content-Type': 'text/html;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Location': '/web/20181112223839/https://www.gidsvoornederland.nl/werken-met-gids/meerwaarde-voor-bibliotheken/bibliotheken-in-nederland?pi=364&organisation_id=199398&startindex=1010', 'Set-Cookie': 'JSESSIONID=4B2D56A6E6715B80C11D3478467B90C7; Path=/; HttpOnly', 'X-Archive-Orig-Vary': 'Accept-Encoding,User-Agent', 'X-Archive-Guessed-Charset': 'utf-8', 'X-Archive-Orig-Pragma': 'no-cache', 'X-Archive-Orig-Server': 'Apache/2.2.22 (Ubuntu)', 'X-Archive-Orig-Connection': 'close', 'X-Archive-Orig-Content-Type': 'text/html', 'X-Archive-Orig-X-Powered-By': 'PHP/5.3.10-1ubuntu3.26', 'X-Archive-Orig-Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pr